In [1]:
import pandas as pd
import numpy as np
import arxivscraper
from random import randint
import matplotlib.pyplot as plt
import datetime
import json

#### Make authors, papers and authors_paper without rules

WARNING: Running this cell only if you want to extract all the data again, it takes too long

In [2]:
# Getting data computer science session
if True:
    scraper_cs = arxivscraper.Scraper(category='math', date_from='2007-01-01',date_until='2009-12-30')
    output = scraper_cs.scrape()

    with open('../datasets/_cs_scraper.json', 'w') as fp:
        json.dump(output, fp)

#### Download dataset

#### ** Step initial **
WARNING: This cell should only be executed if it does not have the data on the local machine

In [4]:
if False:
    !wget https://www.dropbox.com/s/s5mysx0b6sp7jvw/_authors.json -P ../datasets/
    !wget https://www.dropbox.com/s/k6knmy7pvqiqo8v/_author_paper.json -P ../datasets/
    !wget https://www.dropbox.com/s/z5qz0al2eezdlin/_papers.json -P ../datasets/

In [3]:
if True:
    
    with open('../datasets/_cs_scraper.json') as fp:
        output = json.loads(fp.read())
        
    authors = []

    for aut in output:
        for i, a in enumerate(aut['authors']):
            authors.append({'idauthor': f'auth_{(i + randint(1,21)*5)}_{(i + randint(1,21))}', 'name': a})

    authors
    
    #Generate papers
    papers = []

    for aut in output:
        papers.append({'idpaper': aut['id'], 'title': aut['title'], 'abstract': aut['abstract'], 'date_create': aut['created'], 'area': aut['categories'], 'authors': aut['authors']})

    papers
    
    # Generate authors_paper
    author_paper = []

    for p in papers:
        for a in authors:
            if a['name'] in p['authors']:
                author_paper.append({'author_idauthor': a['idauthor'], 'paper_idpaper': p['idpaper'], 'date_create': p['date_create']})
    author_paper
    
    # Save .json files
    import json
    with open('../datasets/_authors.json', 'w') as fp:
        json.dump(authors, fp)

    with open('../datasets/_papers.json', 'w') as fp:
        json.dump(papers, fp)

    with open('../datasets/_author_paper.json', 'w') as fp:
        json.dump(author_paper, fp)

#### Apply rules

1. Remove papers with only one author
2. All areas that are in year 1 must be in year 2
3. Remove authors who appear in year 1 and do not appear in year 2
4. Remove authors who are only in year 2

In [5]:
if True:
    # load dicts
    import json
    with open('../datasets/_authors.json') as fp:
        authors = json.loads(fp.read())

    with open('../datasets/_papers.json') as fp:
        papers = json.loads(fp.read())

    # drop single authors
    papers_with_multiple_authors = []

    for p in papers:
        if len(p['authors']) > 1:
            papers_with_multiple_authors.append(p)

    papers = papers_with_multiple_authors

    # Generate authors_paper
    author_paper = []

    for p in papers:
        for a in authors:
            if a['name'] in p['authors']:
                author_paper.append({'author_idauthor': a['idauthor'], 'paper_idpaper': p['idpaper'], 'date_create': p['date_create']})
    len(author_paper)

    # Save .json files
    import json
    with open('../datasets/_authors.json', 'w') as fp:
        json.dump(authors, fp)

    with open('../datasets/_papers.json', 'w') as fp:
        json.dump(papers, fp)

    with open('../datasets/_author_paper.json', 'w') as fp:
        json.dump(author_paper, fp)

#### Load dataframes authors

In [7]:
df_authors = pd.read_json('../datasets/_authors.json')
df_authors.drop_duplicates(subset=['idauthor'], keep=False, inplace=True)
df_authors.head()

,idauthor,name
1131,auth_106_26,lasenby
1132,auth_32_27,martinez-gonzalez
1229,auth_84_23,poor
1626,auth_69_6,lotker
3258,auth_105_22,labourie


#### Load dataframes author_paper

In [40]:
df_author_paper = pd.read_json('../datasets/_author_paper.json')
#df_author_paper.drop_duplicates(subset=['date_create', 'paper_idpaper'], keep=False, inplace=True)
df_filter = df_author_paper[df_author_paper['date_create'] > '2004-01-01']
df_filter = df_filter[df_filter['date_create'] < '2007-01-01']

In [42]:
df_filter.head()

,author_idauthor,date_create,paper_idpaper
1716429,auth_85_1,2004-02-12,astro-ph/0402324
1716430,auth_106_12,2004-02-12,astro-ph/0402324
1716431,auth_100_13,2004-02-12,astro-ph/0402324
1716432,auth_55_8,2004-02-12,astro-ph/0402324
1716433,auth_81_17,2004-02-12,astro-ph/0402324


In [26]:
df_author_paper.head()

,author_idauthor,date_create,paper_idpaper
0,auth_35_17,2007-03-30,0704.0002
1,auth_41_7,2007-03-30,0704.0002
2,auth_100_17,2007-03-30,0704.0002
3,auth_61_5,2007-03-30,0704.0002
4,auth_56_10,2007-03-30,0704.0002


In [29]:
len(df_author_paper)

4250129

#### Load dataframes paper

In [9]:
df_paper = pd.read_json('../datasets/_papers.json')
df_paper.drop(columns=['authors'], axis=1, inplace=True)
df_paper.drop_duplicates(subset=['area', 'idpaper', 'title'], keep=False, inplace=True)
df_paper.tail()

,abstract,area,date_create,idpaper,title
43813,a n=4 supersymmetric matrix kp hierarchy is pr...,solv-int hep-th math-ph math.mp nlin.si,1999-06-29,solv-int/9907004,n=2 local and n=4 nonlocal reductions of super...
43814,we introduce a class of cellular automata asso...,solv-int math.qa nlin.si,1999-07-27,solv-int/9907020,soliton cellular automata associated with crys...
43815,for the integrable case of the discrete self-t...,solv-int hep-th math.ca math.qa nlin.si,1999-07-31,solv-int/9908002,quantum backlund transformation for the integr...
43816,a family of completely integrable nonlinear de...,solv-int math.qa nlin.si,1999-11-03,solv-int/9911004,integrable deformations of oscillator chains f...
43817,three inequivalent real forms of the complex n...,solv-int hep-th math-ph math.mp nlin.si,1999-11-04,solv-int/9911005,a note on real forms of the complex n=4 supers...
